<a href="https://colab.research.google.com/github/Jio-Kim/ML-DL/blob/main/LogisticRegression(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 소프트맥스(다중 분류 로지스틱 회귀)

- M개의 입력을 받아 N개의 클래스로 분류하는 로지스틱 회귀 모델을 케라스로 구현
- 보통 다중 분류 로지스틱 회귀 모델을 소프트맥스라고 부름

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

### 데이터 획득

In [7]:
# MNIST 손글씨 데이터를 내려받아 변수에 저장
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [8]:
print("train data(count, row, column) : " + str(X_train.shape))
print("test data(count, row, column) : " + str(X_test.shape)) 

train data(count, row, column) : (60000, 28, 28)
test data(count, row, column) : (10000, 28, 28)


### 데이터 정규화
- 모델 학습에 앞서 데이터를 정규화
- 정규화는 입력값을 0부터 1의 값으로 변경
- 정규화된 입력값은 경사하강법으로 모델을 학습할 때 더욱 쉽고 빠르게 최적의 w,b를 찾도록 도와줌

In [9]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# 데이터 정규화
X_train /= 255
X_test /=255

- y_train, y_test는 손글씨 데이터에 해당하는 숫자
- y_train에는 6만개, y_test에는 1만개의 숫자 담겨 있음

In [10]:
print('train target (count) : ' + str(Y_train.shape))
print('test target (count) : ' + str(Y_test.shape))

train target (count) : (60000,)
test target (count) : (10000,)


In [11]:
# y_train, y_test 샘플로 숫자 출력
print('train target (count) : ' + str(Y_train[0]))
print('test target (count) : ' + str(Y_test[0]))

train target (count) : 5
test target (count) : 7


### 데이터 단순화
- 28 * 28 픽셀의 행/열 지역적인 정보를 사용하지 않고 단순히 정규화된 입력값만 가지고 숫자를 분류할 것이므로 행과 열의 구분 없이 단순히 784(28*28) 길이의 배열로 데이터 단순화

In [13]:
input_dim = 784
X_train = X_train.reshape(60000, input_dim)
X_test = X_test.reshape(10000, input_dim)

In [16]:
# 현재 데이터가 2차원이 아닌 1차원으로 변경된 것 확인 가능
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


### 소프트맥스
- 소프트맥스는 정규화된 여러 개의 로지스틱 회귀로 구성되어 있으며, 10개의 로지스틱 회귀를 배열로 나타낼 경우 [L0, L1,L2,L3,L4,L5,L6,L7,L8,L9]로 나타낼 수 있음
- 각 인덱스는 각 숫자를 의미함


- 로지스틱 회귀이기 때문에 각 L의 값은 0~1이며, 만약 출력이 [0.9,0.1,0,0,0,0,0,0,0]인 경우 가장 높은 확률을 가진 첫번쨰 인덱스, 즉 0이 소프트맥스의 출력값이 됨

In [18]:
# 학습시 실제값과의 크로스 엔트로피를 계산해야 하므로 아래 코드를 실행해 실제값을 원 핫 인코딩으로 변환
num_classes = 10
y_train = to_categorical(Y_train, num_classes)
y_test = to_categorical(Y_test, num_classes)

In [19]:
print(y_train[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [20]:
# 케라스의 Sequential()을 사용하면 간단하게 소프트맥스 구현 가능
# 784개의 입력을 받아 10개의 시그모이드 값을 출력
model = Sequential()
model.add(Dense(input_dim = input_dim, units = 10, activation = 'softmax'))

In [26]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy']) 
model.fit(X_train, y_train, batch_size=2048, epochs=100, verbose=0) 

### 모델 테스트

In [27]:
score = model.evaluate(X_test, y_test)
print('Test accuracy":', score[1])

313/313 [==============================] - 1s 1ms/step - loss: 0.4221 - accuracy: 0.8914
Test accuracy": 0.8913999795913696


### 모델 요약
- 총 10개의 로지스틱 회귀, 각 로지스틱 회귀는 784개의 회귀계수(W)와 1개의 편향(bias)가 있어 총 7850(785*10)개의 param 존재

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                7850      
                                                                 
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [29]:
# 회귀계수의 개수와 편향의 개수 확인
model.layers[0].weights

[<tf.Variable 'dense/kernel:0' shape=(784, 10) dtype=float32, numpy=
 array([[ 0.01563162, -0.00993092,  0.022488  , ..., -0.03123334,
          0.07233564, -0.05679205],
        [ 0.04933815, -0.04028556, -0.011798  , ...,  0.02394254,
          0.06982256, -0.07205341],
        [ 0.07182535, -0.06674766,  0.02926891, ...,  0.07262319,
         -0.02922601,  0.05645695],
        ...,
        [-0.07869192,  0.0583502 ,  0.04053553, ..., -0.0775377 ,
         -0.05193917,  0.05881495],
        [ 0.06080773, -0.00988601, -0.03853075, ..., -0.0740303 ,
         -0.03198742, -0.06326766],
        [ 0.01619154, -0.0532421 ,  0.0563123 , ...,  0.05487303,
          0.07253354, -0.0078078 ]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=
 array([-0.07476708,  0.1559597 , -0.02923649, -0.0637788 ,  0.048158  ,
         0.14809175, -0.02139518,  0.10115607, -0.23789537, -0.02629288],
       dtype=float32)>]